#<b> Triggers </b>

<ul>
<li> Un trigger es un objeto que se ejecuta automáticamente cuando los datos de una tabla han cambiado (INSERT | DELETE | UPDATE).</li>
<li> Pueden ser utilizados para auditar operaciones sobre datos sensibles de la BD y registrar valores asociados a dichas operaciones.</li>
<li> Permiten al DBMS asegurar ciertas reglas/valores asociados a los datos. Por ejemplo, generar valores para columnas derivadas.</li>
<li> Si se elimina la tabla, también se eliminarán los triggers que tenga asociados.</li>
</ul>

#<b> Triggers y SQLite </b>

```SQL
CREATE TRIGGER [IF NOT EXISTS] trigger_name
   [BEFORE|AFTER] [INSERT|UPDATE|DELETE]
   ON table_name
   [WHEN condition]
BEGIN
 statements;
END;
);
```

<center><img src="https://drive.google.com/uc?id=10sJfcsiXChh1F7kE0-Se7biDnoSVWwea"></center>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("/content/drive/MyDrive/06 SQL/C07/Chinook_Sqlite.sqlite")
cursor = conn.cursor()

<center><img src="https://drive.google.com/uc?id=1fKhtlmyFD2Vryk3ubFX4g6FsS0PNLNaZ" alt="drawing" width=700px></center>

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';") #Seleccionamos la columna Nombre de la tabla sqlite_master, donde la columna type sea igual a 'table'
print(cursor.fetchall()) #Le pedimos a nuestro cursor que obtenga todos los resultados y los imprima:

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]


# <b> CASO 1: Registrando (log) cambios en los datos. </b>
Volveremos sobre el caso del descuento en el precio de los tracks y crearemos una tabla para registrar los descuentos:
```SQL
CREATE TABLE [TrackLog]
(
    [TrackId] INTEGER  NOT NULL,
    [Name] NVARCHAR(200)  NOT NULL,
    [AlbumId] INTEGER,
    [MediaTypeId] INTEGER  NOT NULL,
    [GenreId] INTEGER,
    [Composer] NVARCHAR(220),
    [Milliseconds] INTEGER  NOT NULL,
    [Bytes] INTEGER,
    [UnitPrice] NUMERIC(10,2)  NOT NULL,
    CONSTRAINT [PK_Track] PRIMARY KEY  ([TrackId])
)
```


In [ ]:
#Creamos la tabla Track_BKP
sql_TrackLog = """
drop table if exists TrackLog;
"""
cursor.execute(sql_TrackLog)
conn.commit()

In [ ]:
#Creamos la tabla Track_BKP
sql_TrackLog = """
CREATE TABLE [TrackLog]
(
    [TrackId] INTEGER  NOT NULL,
    [Name] NVARCHAR(200),
    [AlbumId] INTEGER,
    [MediaTypeId] INTEGER  NOT NULL,
    [GenreId] INTEGER,
    [Composer] NVARCHAR(220),
    [Milliseconds] INTEGER  NOT NULL,
    [Bytes] INTEGER,
    [UnitPrice] NUMERIC(10,2)  NOT NULL,
    [Timestamp] TEXT NOT NULL,
    CONSTRAINT [PK_Track] PRIMARY KEY  ([TrackId])
)
"""
cursor.execute(sql_TrackLog)
conn.commit()

In [ ]:
pd.read_sql("SELECT * FROM TrackLog",conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,Timestamp


Si el administrador decide hacer un descuento sobre los temas de algunos grupos, un trigger (AFTER UPDATE) registrará, en la tabla TrackLog, el precio unitario de cada tema antes del descuento junto con una marca de tiempo:

In [ ]:
sql_trigger = """
             drop trigger if exists logDiscount;
"""
cursor.executescript(sql_trigger)
conn.commit()

In [ ]:
sql_trigger = """
              CREATE TRIGGER logDiscount
              AFTER UPDATE
              ON Track
              WHEN old.UnitPrice != new.UnitPrice

              BEGIN
                   INSERT INTO TrackLog values(old.TrackId, old.Name, old.AlbumId,
                                               old.MediaTypeId, old.GenreId, old.Composer,
                                               old.Milliseconds, old.Bytes, old.UnitPrice, datetime('now'));
              END;
"""
cursor.executescript(sql_trigger)
conn.commit()

In [ ]:
sqlQuery = "SELECT  DATETIME('NOW')"
pd.read_sql(sqlQuery,conn)

,DATETIME('NOW')
0,2025-03-31 19:46:02


Ahora escribiremos la consulta para hacer el descuento de un $50\%$ en el precio de los temas de AC/DC y Metallica:

In [ ]:
subSqlQuery = """
        SELECT t.TrackId from Track T
        INNER JOIN Album al
        ON t.AlbumId = al.AlbumId
        INNER JOIN Artist a
        ON al.ArtistId = a.ArtistId
        WHERE a.Name IN ('AC/DC','Metallica')
"""

In [ ]:
sqlQuery2 = f"""
UPDATE Track
    SET UnitPrice = UnitPrice * 0.5
    WHERE Track.trackId IN
    ({subSqlQuery})

"""

In [ ]:
cursor.execute(sqlQuery2)
conn.commit()

Verificamos el contenido en las tablas Track y TrackLog:

In [ ]:
sqlQuery = "SELECT * FROM Track Where UnitPrice < 0.99 "
pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.495
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.495
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.495
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.495
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.495
...,...,...,...,...,...,...,...,...,...
125,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.495
126,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.495
127,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.495
128,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.495


In [ ]:
sqlQuery = "SELECT * FROM TrackLog"
pd.read_sql(sqlQuery,conn)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,Timestamp
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,2025-03-31 19:52:42
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,2025-03-31 19:52:42
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99,2025-03-31 19:52:42
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,2025-03-31 19:52:42
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,2025-03-31 19:52:42
...,...,...,...,...,...,...,...,...,...,...
125,1897,The Shortest Straw,156,1,3,James Hetfield and Lars Ulrich,395389,13013990,0.99,2025-03-31 19:52:42
126,1898,Harvester Of Sorrow,156,1,3,James Hetfield and Lars Ulrich,345547,11377339,0.99,2025-03-31 19:52:42
127,1899,The Frayed Ends Of Sanity,156,1,3,"James Hetfield, Lars Ulrich and Kirk Hammett",464039,15198986,0.99,2025-03-31 19:52:42
128,1900,To Live Is To Die,156,1,3,"James Hetfield, Lars Ulrich and Cliff Burton",588564,19243795,0.99,2025-03-31 19:52:42


#<b>CASO 2: Validando datos </b>
Usaremos un trigger (BEFORE INSERT) para validar la dirección de correo de un nuevo cliente.

In [ ]:
sql_trigger = """
             drop trigger if exists validateEmail;
"""
#sql_trigger=""
cursor.executescript(sql_trigger)
conn.commit()

In [ ]:
sql_trigger = """
              CREATE TRIGGER validateEmail
              BEFORE INSERT
              ON Customer

              BEGIN
                   SELECT
                         CASE
	                           WHEN NEW.email NOT LIKE '%_@__%.__%' THEN
   	                             RAISE (ABORT,'Invalid email address')
                         END;
              END;
"""
#sql_trigger=""
cursor.executescript(sql_trigger)
conn.commit()

In [ ]:
sql_Customer = pd.read_sql("SELECT * FROM sqlite_master WHERE type='table' AND name = 'Customer';",conn).sql[0]
print(sql_Customer)

CREATE TABLE [Customer]
(
    [CustomerId] INTEGER  NOT NULL,
    [FirstName] NVARCHAR(40)  NOT NULL,
    [LastName] NVARCHAR(20)  NOT NULL,
    [Company] NVARCHAR(80),
    [Address] NVARCHAR(70),
    [City] NVARCHAR(40),
    [State] NVARCHAR(40),
    [Country] NVARCHAR(40),
    [PostalCode] NVARCHAR(10),
    [Phone] NVARCHAR(24),
    [Fax] NVARCHAR(24),
    [Email] NVARCHAR(60)  NOT NULL,
    [SupportRepId] INTEGER,
    CONSTRAINT [PK_Customer] PRIMARY KEY  ([CustomerId]),
    FOREIGN KEY ([SupportRepId]) REFERENCES [Employee] ([EmployeeId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


In [ ]:
sql_Customer = "select max(CustomerId) from Customer;"
pd.read_sql(sql_Customer,conn)

,max(CustomerId)
0,59


Ahora agregaremos un nuevo cliente:

In [ ]:
sqlQuery = "INSERT INTO Customer values(60,'Pepe','Lota','Aplaplac','Avda Ines Matte','Santiago','XXXX','Chile','XXXX','+56912345678','---','pepe_aplaplac.cl',8)"
cursor.execute(sqlQuery)

IntegrityError: Invalid email address

In [ ]:
sqlQuery = "INSERT INTO Customer values(60,'Pepe','Lota','Aplaplac','Avda Ines Matte','Santiago','XXXX','Chile','XXXX','+56912345678','---','pepe@aplaplac.cl',8)"
cursor.execute(sqlQuery)

In [ ]:
sql_Customer = "select max(CustomerId) from Customer;"
pd.read_sql(sql_Customer,conn)

,max(CustomerId)
0,60


#<b>CASO 3: Actualizando atributos derivados </b>
Consideremos una versión simplificada de una BD de una empresa constituida por departamentos y sus empleados. Para cada empleado se registra su RUT, Nombre, Salario y Supervisor. Para cada departamento se registra su Nombre, jefe y el salario de todos los empleados que pertenecen a él. Este último atributo es un atributo derivado que corresponde a la suma de los salarios de todos los empleados que trabajan en un mismo departamento.

**Evento (a) Inserción de uno o más registros de empleados.** En este caso, luego de la inserción de los datos de un empleado, es necesario actualizar el atributo derivado `SalarioTotal` de la entidad `Departamento`.<br>
**Evento (b) Cambio en el salario de uno o más empleados**. En este caso, luego de la actualización del salario de un empleado, es necesario actualizar el atributo derivado `SalarioTotal` de la entidad `Departamento`.<br>
**Evento (c) Unos o más empleados son asignados a otro departamento**. En este caso, luego que un empleado es asignado a otro departamento, es necesario actualizar el atributo derivado `SalarioTotal` de la entidad `Departamento` tanto en el departamento de origen como en el departamento de destino.<br>
**Evento (d) Unos o más empleados ya no son parte de la empresa**. En este caso, luego que un empleado ya no es parte de la empresa el atributo derivado `SalarioTotal` de la entidad `Departamento` debe ser actualizado.<br>

El MER para este caso es el siguiente:



<center><img src="https://drive.google.com/uc?id=1dGdFFt9-GCKIwNShFVxix6sja_6fvh4u" alt="drawing" width=700px></center>

y el Modelo relacional es el siguiente:

```
 CREATE TABLE Departamento
(
  DeptID VARCHAR(10) NOT NULL AUTOINCREMENT,
  Nombre VARCHAR(30) NOT NULL,
  Jefe VARCHAR(10) NOT NULL,
  SalarioTotal INT NOT NULL,
  PRIMARY KEY (DeptID)
);

CREATE TABLE Empleado
(
  Nombre VARCHAR(30) NOT NULL,
  Rut VARCHAR(10) NOT NULL,
  Salario INT NOT NULL,
  Supervisor VARCHAR(10),
  DeptID VARCHAR(10),
  PRIMARY KEY (Rut),
  FOREIGN KEY (DeptID) REFERENCES Departamento(DeptID)
);
```

Las siguientes sentencias insertan registros en las tablas `Departamento` y  `Empleado` respectivamente:

```
INSERT INTO Departamento (Nombre, Jefe, SalarioTotal)
VALUES ("Marketing", "4917481-0",589),
       ("Informática", "40391531-9",1697);
```

```
INSERT INTO Empleado (Nombre, Rut, Salario, Supervisor, DeptID)
VALUES
  ("Ezekiel Marquez","48123222-8",126,"4917481-0",1),
  ("Alika Craig","27524290-K",463,"4917481-0",1),
  ("Dawn Gilliam","4917481-0",805,,2),
  ("Clinton Harrison","37707378-9",196,"40391531-9",2),
  ("Clinton Browning","40391531-9",696,,2);
```

Ahora crearemos la BD usando el script `EmpDept.sql`:

In [ ]:
#Importamos la librería SQLITE3, para utilizar SQL
import sqlite3
import pandas as pd
#Creamos una conexión que nos permite crear un entorno de base de datos.
conn = sqlite3.connect("database.db") #Creamos una base de datos a partir del método connect()
#Creamos un cursor que nos permite apuntar a la conección de nuestra Base de Datos
cursor = conn.cursor()
#Abrir archivo con las sentencias para la creación de tablas.
sqlScript = open("/content/drive/MyDrive/06 SQL/C07/EmpDept.sql").read()
#Ejecutamos el script que contiene la creación/modificación de tablas
cursor.executescript(sqlScript)

In [ ]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';",conn)

,name
0,Departamento
1,sqlite_sequence
2,Empleado


Insertamos registros en las tablas `Departameno` y `Empleado` respectivamente. Notar que la columna `SalarioTotal` del departamento de `Marketing` es equivalente a la suma de los salarios de todos los empleados de dicho departamento. Lo mismo sucede se verifica para los datos asociados al departamento de `Informática`.

In [ ]:
sqlInsert = """
INSERT INTO Departamento (Nombre, Jefe, SalarioTotal)
VALUES ("Marketing", "4917481-0",589),
       ("Informática", "40391531-9",1697);
"""
cursor.executescript(sqlInsert)

In [ ]:
pd.read_sql("SELECT * FROM Departamento",conn)

,DeptID,Nombre,Jefe,SalarioTotal
0,1,Marketing,4917481-0,589
1,2,Informática,40391531-9,1697


In [ ]:
sqlInsert = """
INSERT INTO Empleado (Nombre, Rut, Salario, Supervisor, DeptID)
VALUES
  ("Ezekiel Marquez","48123222-8",126,"4917481-0",1),
  ("Alika Craig","27524290-K",463,"4917481-0",1),
  ("Dawn Gilliam","4917481-0",805,NULL,2),
  ("Clinton Harrison","37707378-9",196,"40391531-9",2),
  ("Clinton Browning","40391531-9",696,NULL,2);
"""
cursor.executescript(sqlInsert)

In [ ]:
pd.read_sql("SELECT * FROM Empleado",conn)

,Nombre,Rut,Salario,Supervisor,DeptID
0,Ezekiel Marquez,48123222-8,126,4917481-0,1
1,Alika Craig,27524290-K,463,4917481-0,1
2,Dawn Gilliam,4917481-0,805,None,2
3,Clinton Harrison,37707378-9,196,40391531-9,2
4,Clinton Browning,40391531-9,696,None,2


##**Evento (a) Inserción de uno o más registros de empleados.**



1.   Crearemos el trigger que se ejecutará luego de la inserción de un registro en la tabla `Empleado`.
2.   Agregaremos un nuevo empleado al departamento de `Marketing`.
3.   Verificaremos que la columna `SalarioTotal` se actualiza correctamente.



###**1. Creación del trigger.**

In [ ]:
sqlTrigger = """
            CREATE TRIGGER CalculateTotalSalaryA
            AFTER INSERT ON Empleado
            BEGIN
              UPDATE Departamento
              SET SalarioTotal = SalarioTotal + NEW.Salario
              WHERE DeptID = NEW.DeptID;
            END;
"""
cursor.executescript(sqlTrigger)

###**2. Agregar un nuevo empleado.**

In [ ]:
insert = """
INSERT INTO Empleado (Nombre, Rut, Salario, Supervisor, DeptID)
VALUES
  ("Mira Lindsay","48123222-7",11,"4917481-0",1);
  """
cursor.executescript(insert)

In [ ]:
pd.read_sql("SELECT * FROM Empleado",conn)

,Nombre,Rut,Salario,Supervisor,DeptID
0,Ezekiel Marquez,48123222-8,126,4917481-0,1
1,Alika Craig,27524290-K,463,4917481-0,1
2,Dawn Gilliam,4917481-0,805,None,2
3,Clinton Harrison,37707378-9,196,40391531-9,2
4,Clinton Browning,40391531-9,696,None,2
5,Mira Lindsay,48123222-7,11,4917481-0,1


###**3. Verificamos que la columna SalarioTotal se actualice correctamente.**

In [ ]:
pd.read_sql("SELECT * FROM Departamento",conn)

,DeptID,Nombre,Jefe,SalarioTotal
0,1,Marketing,4917481-0,600
1,2,Informática,40391531-9,1697


#**Activivad 1**

De manera similar a lo mostrado con el Evento (a): implemente un trigger, active su ejecución con una sentencia de insert o update y posteriormente verifique usando select para cada uno de los siguiente eventos:

*   Evento (b) Cambio en el salario de uno o más empleados.
*   Evento (c) Uno o más empleados son asignados a otro departamento.
*   Evento (d) Uno o más empleados ya no son parte de la empresa.



###**Evento (b)**

In [ ]:
# Su código aquí
sqlTrigger = """
            CREATE TRIGGER CalculateTotalSalaryB
            AFTER UPDATE ON Empleado
            WHEN old.Salario != new.Salario
            BEGIN
              UPDATE Departamento
              SET SalarioTotal = SalarioTotal + NEW.Salario - OLD.Salario
              WHERE DeptID = NEW.DeptID;
            END;
"""
cursor.executescript(sqlTrigger)

In [ ]:
# Activar Ejecucion
cambio_salario = """
  UPDATE Empleado
  SET Salario = 396
  WHERE Rut = "37707378-9";
  """
cursor.executescript(cambio_salario)

In [ ]:
# Verificar
pd.read_sql("SELECT * FROM Empleado",conn)

,Nombre,Rut,Salario,Supervisor,DeptID
0,Ezekiel Marquez,48123222-8,126,4917481-0,1
1,Alika Craig,27524290-K,463,4917481-0,1
2,Dawn Gilliam,4917481-0,805,None,2
3,Clinton Harrison,37707378-9,396,40391531-9,2
4,Clinton Browning,40391531-9,696,None,2
5,Mira Lindsay,48123222-7,11,4917481-0,1


In [ ]:
# Verificar
pd.read_sql("SELECT * FROM Departamento",conn)

,DeptID,Nombre,Jefe,SalarioTotal
0,1,Marketing,4917481-0,600
1,2,Informática,40391531-9,1897


El trabajador del Departamento de Informática aumentó su sueldo en 200 unidades, y el Salario Total de ese Departamento se modificó consecuentemente.

###**Evento (c)**

In [ ]:
# Su código aquí
sqlTrigger = """
            CREATE TRIGGER CalculateTotalSalaryC
            AFTER UPDATE ON Empleado
            WHEN old.DeptID != new.DeptID
            BEGIN
              UPDATE Departamento
              SET SalarioTotal = SalarioTotal + NEW.Salario
              WHERE DeptID = NEW.DeptID;
              UPDATE Departamento
              SET SalarioTotal = SalarioTotal - OLD.Salario
              WHERE DeptID = OLD.DeptID;
            END;
"""
cursor.executescript(sqlTrigger)

In [ ]:
# Activar Ejecucion
cambio_departamento = """
  UPDATE Empleado
  SET DeptID = 1
  WHERE Rut = "37707378-9";
  """
cursor.executescript(cambio_departamento)

In [ ]:
# Verificar
pd.read_sql("SELECT * FROM Empleado",conn)

,Nombre,Rut,Salario,Supervisor,DeptID
0,Ezekiel Marquez,48123222-8,126,4917481-0,1
1,Alika Craig,27524290-K,463,4917481-0,1
2,Dawn Gilliam,4917481-0,805,None,2
3,Clinton Harrison,37707378-9,396,40391531-9,1
4,Clinton Browning,40391531-9,696,None,2
5,Mira Lindsay,48123222-7,11,4917481-0,1


In [ ]:
# Verificar
pd.read_sql("SELECT * FROM Departamento",conn)

,DeptID,Nombre,Jefe,SalarioTotal
0,1,Marketing,4917481-0,996
1,2,Informática,40391531-9,1501


El trabajador cambió desde el Departamento de Informática a Marketing, y el Salario Total de cada Departamento se modificó consecuentemente.

###**Evento (d)**

In [ ]:
# Su código aquí
sqlTrigger = """
            CREATE TRIGGER CalculateTotalSalaryD
            AFTER DELETE ON Empleado
            BEGIN
              UPDATE Departamento
              SET SalarioTotal = SalarioTotal - OLD.Salario
              WHERE DeptID = OLD.DeptID;
            END;
"""
cursor.executescript(sqlTrigger)

In [ ]:
# Activar Ejecucion
salida_trabajador = """
  DELETE FROM Empleado
  WHERE Rut = "37707378-9";
  """
cursor.executescript(salida_trabajador)

In [ ]:
# Verificar
pd.read_sql("SELECT * FROM Empleado",conn)

,Nombre,Rut,Salario,Supervisor,DeptID
0,Ezekiel Marquez,48123222-8,126,4917481-0,1
1,Alika Craig,27524290-K,463,4917481-0,1
2,Dawn Gilliam,4917481-0,805,None,2
3,Clinton Browning,40391531-9,696,None,2
4,Mira Lindsay,48123222-7,11,4917481-0,1


In [ ]:
# Verificar
pd.read_sql("SELECT * FROM Departamento",conn)

,DeptID,Nombre,Jefe,SalarioTotal
0,1,Marketing,4917481-0,600
1,2,Informática,40391531-9,1501


Se ejecuta la salida de un trabajador de Marketing, y el SalarioTotal de este departamento se ajusta consecuentemente.

#**Fin Activivad 1**

# <font color='red'>__LINK DE INTERÉS__: Trigger en SQLite</font>

El siguiente link se aborda el uso de Triggers en SQLite, para automatizar el envío de correos electrónicos a nuevos usuarios de una Plataforma [aquí](https://turso.tech/blog/use-sqlite-triggers-to-send-new-user-emails).